In [1]:
import elo

In [2]:
test = elo.ELO(player_list=['Ryan','Sean','Michael'])

In [3]:
test.EM.show_cache()

{'Ryan': 1600, 'Sean': 1600, 'Michael': 1600}


In [4]:
test.do_competition(winner='Ryan',loser='Sean')
test.EM.show_cache()

{'Ryan': 1616.0, 'Sean': 1584.0, 'Michael': 1600}
